In [1]:
import sys
from pathlib import Path   
root_dir = str(Path().absolute().parent)

if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Adding the following directory to the PYTHONPATH: d:\Projects\ML\personalized-recommendation-sysyem\personalized-recommendation-system


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from recsys import hopsworks_integration
from recsys.config import settings
from recsys.hopsworks_integration import feature_store

d:\Projects\ML\personalized-recommendation-sysyem\personalized-recommendation-system\uvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Conect to hopswork feature store
project, fs = feature_store.get_feature_store()

2025-03-15 16:12:02.090 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-03-15 16:12:02,093 INFO: Initializing external client
2025-03-15 16:12:02,094 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-15 16:12:05,048 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213659


In [5]:
from loguru import logger
import polars as pl

In [45]:
customers_df= pl.read_csv("../data/customers.csv")

In [46]:
customers_df.shape

(1371980, 7)

In [47]:
customers_df.head()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"
"""00005ca1c9ed5f5146b52ac8639a40…",null,null,"""ACTIVE""","""NONE""",54,"""5d36574f52495e81f019b680c843c4…"
"""00006413d8573cd20ed7128e53b7b1…",1.0,1.0,"""ACTIVE""","""Regularly""",52,"""25fa5ddee9aac01b35208d01736e57…"


In [48]:
customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


In [49]:
from recsys.features.customers import DatasetSampler, compute_features_customers

In [50]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [51]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""


In [55]:
import pandas as pd

# Check for missing values
# print(customers_df.isnull().sum())
print(customers_df.null_count())

shape: (1, 5)
┌─────────────┬────────────────────┬─────┬─────────────┬───────────┐
│ customer_id ┆ club_member_status ┆ age ┆ postal_code ┆ age_group │
│ ---         ┆ ---                ┆ --- ┆ ---         ┆ ---       │
│ u32         ┆ u32                ┆ u32 ┆ u32         ┆ u32       │
╞═════════════╪════════════════════╪═════╪═════════════╪═══════════╡
│ 0           ┆ 0                  ┆ 0   ┆ 0           ┆ 0         │
└─────────────┴────────────────────┴─────┴─────────────┴───────────┘


In [56]:
customers_df.shape

(1356119, 5)

In [15]:
logger.info("Uploading 'customers' Feature Group to Hopsworks.")
customers_fg = feature_store.create_customers_feature_group(
    fs, df=customers_df, online_enabled=True
)

logger.info("✅ Uploaded 'customers' Feature Group to Hopsworks!")

2025-03-09 14:26:46.371 | INFO     | __main__:<module>:1 - Uploading 'customers' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213659/fs/1201283/fg/1403763


Uploading Dataframe: 100.00% |██████████| Rows 1356119/1356119 | Elapsed Time: 02:48 | Remaining Time: 00:00


Launching job: customers_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213659/jobs/named/customers_1_offline_fg_materialization/executions
2025-03-09 14:30:01,577 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-03-09 14:30:25,039 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-03-09 14:34:01,516 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-03-09 14:34:01,826 INFO: Waiting for log aggregation to finish.
2025-03-09 14:34:25,767 INFO: Execution finished successfully.


2025-03-09 14:34:30.618 | INFO     | __main__:<module>:6 - ✅ Uploaded 'customers' Feature Group to Hopsworks!


In [7]:
transactions_df_all = pl.read_csv("../data/transactions_train.csv")

In [8]:
transactions_df_all.shape

(31788324, 5)

In [ ]:
print(transactions_df_all.null_count())

shape: (1, 5)
┌───────┬─────────────┬────────────┬───────┬──────────────────┐
│ t_dat ┆ customer_id ┆ article_id ┆ price ┆ sales_channel_id │
│ ---   ┆ ---         ┆ ---        ┆ ---   ┆ ---              │
│ u32   ┆ u32         ┆ u32        ┆ u32   ┆ u32              │
╞═══════╪═════════════╪════════════╪═══════╪══════════════════╡
│ 0     ┆ 0           ┆ 0          ┆ 0     ┆ 0                │
└───────┴─────────────┴────────────┴───────┴──────────────────┘


In [12]:
transactions_df = transactions_df_all.head(1500000)

In [13]:
transactions_df_all.head()

t_dat,customer_id,article_id,price,sales_channel_id
str,str,i64,f64,i64
"""2018-09-20""","""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
"""2018-09-20""","""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
"""2018-09-20""","""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2
"""2018-09-20""","""00007d2de826758b65a93dd24ce629…",685687003,0.016932,2
"""2018-09-20""","""00007d2de826758b65a93dd24ce629…",685687004,0.016932,2


In [14]:
print(transactions_df.null_count())

shape: (1, 5)
┌───────┬─────────────┬────────────┬───────┬──────────────────┐
│ t_dat ┆ customer_id ┆ article_id ┆ price ┆ sales_channel_id │
│ ---   ┆ ---         ┆ ---        ┆ ---   ┆ ---              │
│ u32   ┆ u32         ┆ u32        ┆ u32   ┆ u32              │
╞═══════╪═════════════╪════════════╪═══════╪══════════════════╡
│ 0     ┆ 0           ┆ 0          ┆ 0     ┆ 0                │
└───────┴─────────────┴────────────┴───────┴──────────────────┘


In [10]:
transactions_df.shape

(1500000, 5)

In [15]:
from recsys.features import transactions

In [16]:
transactions_df = transactions.compute_features_transactions(transactions_df)

In [17]:
transactions_df.head()

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week,month_sin,month_cos
datetime[ns],str,str,f64,i64,i32,i8,i8,i8,f64,f64
2018-09-20 00:00:00,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4,-1.0,-1.8370e-16
2018-09-20 00:00:00,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4,-1.0,-1.8370e-16
2018-09-20 00:00:00,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4,-1.0,-1.8370e-16
2018-09-20 00:00:00,"""00007d2de826758b65a93dd24ce629…","""685687003""",0.016932,2,2018,9,20,4,-1.0,-1.8370e-16
2018-09-20 00:00:00,"""00007d2de826758b65a93dd24ce629…","""685687004""",0.016932,2,2018,9,20,4,-1.0,-1.8370e-16


In [19]:
transactions_df.null_count()

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week,month_sin,month_cos
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0


In [20]:
logger.info("Uploading 'transactions' Feature Group to Hopsworks.")
trans_fg = feature_store.create_transactions_feature_group(
    fs=fs, df=transactions_df, online_enabled=True
)
logger.info("✅ Uploaded 'transactions' Feature Group to Hopsworks!")

2025-02-27 02:31:55.550 | INFO     | __main__:<module>:1 - Uploading 'transactions' Feature Group to Hopsworks.


⚠️ Warning: month_sin and month_cos already exist in the DataFrame. Skipping transformation.


Uploading Dataframe: 100.00% |██████████| Rows 1500000/1500000 | Elapsed Time: 03:56 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213659/jobs/named/transactions_1_offline_fg_materialization/executions
2025-02-27 02:36:11,663 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-02-27 02:36:15,005 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-02-27 02:39:05,980 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-02-27 02:39:06,245 INFO: Waiting for log aggregation to finish.
2025-02-27 02:39:22,338 INFO: Execution finished successfully.


AttributeError: module 'recsys.hopsworks_integration.constants' has no attribute 'transactions_feature_descriptions'

In [20]:
import torch
from sentence_transformers import SentenceTransformer

In [21]:
from recsys.features.articles import (
    compute_features_articles,
    generate_embeddings_for_dataframe,
)

In [22]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-03-10 19:18:29.902 | INFO     | __main__:<module>:8 - Loading 'all-MiniLM-L6-v2' embedding model to device='cpu'


2025-03-10 19:18:29,920 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [23]:
articles_df= pl.read_csv("../data/articles.csv")

In [24]:
articles_df.head()

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
110065001,110065,"""OP T-shirt (Idro)""",306,"""Bra""","""Underwear""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1339,"""Clean Lingerie""","""B""","""Lingeries/Tights""",1,"""Ladieswear""",61,"""Womens Lingerie""",1017,"""Under-, Nightwear""","""Microfibre T-shirt bra with un…"
110065002,110065,"""OP T-shirt (Idro)""",306,"""Bra""","""Underwear""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1339,"""Clean Lingerie""","""B""","""Lingeries/Tights""",1,"""Ladieswear""",61,"""Womens Lingerie""",1017,"""Under-, Nightwear""","""Microfibre T-shirt bra with un…"


In [25]:
articles_df = compute_features_articles(articles_df)
articles_df.shape

(105542, 27)

In [26]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""10/0108775015.jpg"""
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""10/0108775044.jpg"""
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""10/0108775051.jpg"""


In [28]:
articles_df.null_count()

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2025-03-10 12:02:59.742 | INFO     | __main__:<module>:2 - Item 1:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Dark Black (Black)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-03-10 12:02:59.746 | INFO     | __main__:<module>:2 - Item 2:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Light White (White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-03-10 12:02:59.747 | INFO     | __main__:<module>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
Appearance: Stripe
Color: Dusty Light White (Off White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.


In [35]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)  # Reduce batch size if getting OOM errors.

Generating embeddings: 100%|██████████| 105542/105542 [21:48<00:00, 80.69it/s]


In [36]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.026782, 0.082344, … 0.022782]"
"""Strap top - Vest top in Garmen…","[-0.010396, 0.089874, … 0.022564]"
"""Strap top (1) - Vest top in Ga…","[-0.032753, 0.091124, … 0.022804]"


In [38]:
articles_df = articles_df.select(articles_df.columns[:105500])

In [39]:
logger.info("Uploading 'articles' Feature Group to Hopsworks.")
articles_fg = feature_store.create_articles_feature_group(
    fs,
    df=articles_df,
    articles_description_embedding_dim=model.get_sentence_embedding_dimension(),
    online_enabled=True,
)
logger.info("✅ Uploaded 'articles' Feature Group to Hopsworks!")

2025-03-10 12:35:11.873 | INFO     | __main__:<module>:1 - Uploading 'articles' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 105542/105542 | Elapsed Time: 02:52 | Remaining Time: 00:00


Launching job: articles_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213659/jobs/named/articles_1_offline_fg_materialization/executions
2025-03-10 12:38:27,536 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-03-10 12:38:30,829 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-03-10 12:42:41,638 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-03-10 12:42:41,923 INFO: Waiting for log aggregation to finish.
2025-03-10 12:42:50,927 INFO: Execution finished successfully.


2025-03-10 12:42:51.025 | INFO     | __main__:<module>:8 - ✅ Uploaded 'articles' Feature Group to Hopsworks!


In [26]:
articles_df["image_url"][0]

'10/0108775015.jpg'

In [27]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10px; max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))

Compute ranking dataset

In [6]:
from recsys.features.ranking import compute_ranking_dataset

In [6]:
customers_fg = fs.get_feature_group("customers", version=1)

In [7]:
articles_fg = fs.get_feature_group("articles", version=1)

In [8]:
trans_fg = fs.get_feature_group("transactions", version=1)

In [10]:
ranking_df = compute_ranking_dataset(
    trans_fg,
    articles_fg,
    customers_fg,
)
ranking_df.shape


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (186.72s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (38.73s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (138.14s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1257.05s) 


(14916462, 15)

In [11]:
ranking_df.head(3)

customer_id,age,article_id,label,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
str,f64,str,i32,str,str,str,str,str,str,str,str,str,str,str
"""325b63f3afd49ad98305e8ca136f56…",32.0,"""640727001""",1,"""Skirt""","""Garment Lower body""","""Solid""","""Black""","""Dark""","""Black""","""Skirts""","""Divided""","""Divided""","""Divided Collection""","""Skirts"""
"""5d3190aa6e12099cc77ec23b6817a9…",55.0,"""675657002""",1,"""Dress""","""Garment Full body""","""Solid""","""Dark Red""","""Dark""","""Red""","""Jersey""","""Ladieswear""","""Ladieswear""","""H&M+""","""Jersey Fancy"""
"""139711369b0211ce8ee7be62fa1f24…",30.0,"""633130002""",1,"""Sweater""","""Garment Upper body""","""Colour blocking""","""Dark Blue""","""Dark""","""Blue""","""Tops Fancy Jersey""","""Divided""","""Divided""","""Divided Collection""","""Jersey Fancy"""


In [ ]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
0,13560420
1,1356042


In [ ]:
file_path = r"../data/ranking.csv"  # Change to your desired location

# Save the DataFrame as CSV
ranking_df.write_csv(file_path)


In [9]:
file_path = r"../data/ranking.csv"  

ranking_df = pl.read_csv(file_path)

In [10]:
ranking_sample_df = ranking_df.sample(n=10_000, shuffle=True)

In [11]:
ranking_sample_df.get_column("label").value_counts()

label,count
i64,u32
1,905
0,9095


In [12]:
logger.info("Uploading 'ranking' Feature Group to Hopsworks.")
rank_fg = feature_store.create_ranking_feature_group(
    fs,
    df=ranking_sample_df,
    parents=[articles_fg, customers_fg, trans_fg],
    online_enabled=False
)
logger.info("✅ Uploaded 'ranking' Feature Group to Hopsworks!!")

2025-03-15 16:13:53.309 | INFO     | __main__:<module>:1 - Uploading 'ranking' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 10000/10000 | Elapsed Time: 02:04 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213659/jobs/named/ranking_1_offline_fg_materialization/executions
2025-03-15 16:16:22,219 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-03-15 16:16:25,614 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-03-15 16:18:27,162 INFO: Waiting for log aggregation to finish.
2025-03-15 16:19:28,219 INFO: Execution finished successfully.


2025-03-15 16:19:42.701 | INFO     | __main__:<module>:8 - ✅ Uploaded 'ranking' Feature Group to Hopsworks!!
